<a href="https://colab.research.google.com/github/TharunDD/XGboost/blob/main/xgboost_titanic_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
%matplotlib inline
from xgboost import XGBClassifier
from xgboost import plot_tree
from xgboost import plot_importance

In [ ]:
train_df = pd.read_csv('../input/spaceship-titanic/train.csv')
test_df  = pd.read_csv('../input/spaceship-titanic/test.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.isnull().sum()

In [ ]:
sns.boxplot(x='FoodCourt',data=train_df)

In [ ]:
train_df.loc[train_df['FoodCourt'] > 20000, 'FoodCourt'] = train_df.loc[train_df['FoodCourt'] < 20000, 'FoodCourt'].mean()

In [ ]:
sns.boxplot(x='Transported', y='ShoppingMall',data=train_df)


In [ ]:
train_df.loc[train_df['ShoppingMall'] > 10000, 'ShoppingMall'] = train_df.loc[train_df['ShoppingMall'] < 10000, 'ShoppingMall'].mean()

In [ ]:
sns.boxplot(x='Spa',data=train_df)


In [ ]:
train_df.loc[train_df['Spa'] > 20000, 'Spa'] = train_df.loc[train_df['Spa'] < 20000, 'Spa'].mean()

In [ ]:
sns.boxplot(x='VRDeck',data=train_df)


In [ ]:
train_df.loc[train_df['VRDeck'] > 20000, 'VRDeck'] = train_df.loc[train_df['VRDeck'] < 20000, 'VRDeck'].mean()

In [ ]:
sns.boxplot(x='RoomService',data=train_df)


In [ ]:
train_df.loc[train_df['RoomService'] > 10000, 'RoomService'] = train_df.loc[train_df['RoomService'] < 10000, 'RoomService'].mean()

In [ ]:
train_df.isnull().sum()


In [ ]:
train_df['Age'].fillna(train_df['Age'].mean())
train_df.fillna(method='ffill',inplace=True)
test_df['Age'].fillna(test_df['Age'].mean())
test_df.fillna(method='ffill',inplace=True)

In [ ]:
train_df[['DeckName','DeckNumber','DeckSide']]=train_df['Cabin'].str.split('/',expand=True)
test_df[['DeckName','DeckNumber','DeckSide']]=train_df['Cabin'].str.split('/',expand=True)

In [ ]:
le = preprocessing.LabelEncoder()
def converting_column(df,col,convertType):
    if convertType=='OHE':
        one_hot = pd.get_dummies(df[col],prefix=col)
        df = df.drop(col,axis = 1)
        df = df.join(one_hot)
        return df
    else:
        df[col] = le.fit_transform(df[col])
        return df

In [ ]:
train_df = converting_column(train_df,'HomePlanet','OHE')
test_df = converting_column(test_df,'HomePlanet','OHE')

In [ ]:
train_df = converting_column(train_df,'Destination','FTR')
test_df = converting_column(test_df,'Destination','FTR')

In [ ]:
train_df = converting_column(train_df,'CryoSleep','FTR')
test_df = converting_column(test_df,'CryoSleep','FTR')

In [ ]:
train_df = converting_column(train_df,'VIP','FTR')
test_df = converting_column(test_df,'VIP','FTR')

In [ ]:
train_df['DeckName'] = le.fit_transform(train_df['DeckName'])
train_df['DeckSide'] = le.fit_transform(train_df['DeckSide'])
test_df['DeckName'] = le.fit_transform(test_df['DeckName'])
test_df['DeckSide'] = le.fit_transform(test_df['DeckSide'])

In [ ]:
train_df.head()

In [ ]:
train_df.drop(['PassengerId','Cabin','HomePlanet_Earth','HomePlanet_Europa','HomePlanet_Mars','DeckNumber','Name'],axis=1,inplace=True)
test_df.drop(['PassengerId','Cabin','HomePlanet_Earth','HomePlanet_Europa','HomePlanet_Mars','DeckNumber','Name'],axis=1,inplace=True)


In [ ]:
train_df.corr()


In [ ]:
train_df['kids'] = train_df['Age'].apply(lambda x: 1 if x < 15 else 0)
test_df['kids'] = test_df['Age'].apply(lambda x: 1 if x < 15 else 0)
train_df = train_df.drop(['Age'], axis=1)
test_df = test_df.drop(['Age'], axis=1)

In [ ]:
x_train= train_df.drop("Transported", axis=1)
y_train= train_df["Transported"]

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size = 0.25, random_state =1)

In [ ]:
params = {'learning_rate': 0.13,
          'min_child_weight': 0.15,
          'gamma': 0.25,
          'subsample': 0.65,
          'colsample_bytree': 0.85,
          'colsample_bylevel': 0.64,
          'max_depth': 4,
          'n_estimators': 4500,
          'reg_lambda': 2.25,
          'monotone_constraints': '(0, -1)',
          'use_label_encoder': False
          }
xgb_model = XGBClassifier()
xgb_model.set_params(**params)
mod = xgb_model.fit(X_train, Y_train)

print("Performance on train data:", mod.score(X_train, Y_train))
print(mod.score(X_test,Y_test))

In [ ]:
Y_pred =xgb_model.predict(test_df) == True
submission=pd.read_csv('../input/spaceship-titanic/sample_submission.csv')
submission['Transported'] = Y_pred
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head(20)
